In [ ]:
import requests

url = 'https://feeds.divvybikes.com/stations/stations.json' 

resp = requests.get(url)
resp.raise_for_status()

live_data_raw = resp.json()
live_data_list = live_data_raw.get('stationBeanList')
print(type(live_data), live_data)

In [ ]:
import collections

LiveDataResults = collections.namedtuple(
    'LiveDataResults',
    'id latitude longitude status stationName availableDocks totalDocks availableBikes')



In [ ]:
station_data = []
for station in live_data_list:
     line = LiveDataResults(
        id=station.get('id'),
        latitude=station.get('latitude'),
        longitude=station.get('longitude'),
        status=station.get('status'),
        stationName=station.get('stationName'),
        availableDocks=station.get('availableDocks', 0),
        totalDocks=station.get('totalDocks', 0),
        availableBikes=station.get('availableBikes',0)
     )
     station_data.append(line)


In [ ]:
for i in station_data:
    print(i.id,i.status)

In [ ]:
station_data = []
for station in live_data_list:
        station_dict = {}
        station_dict['station_id']=station.get('id')
        station_dict['latitude']=station.get('latitude')
        station_dict['longitude']=station.get('longitude')
        station_dict['status']=station.get('status')
        station_dict['stationName']=station.get('stationName')
        station_dict['availableDocks']=station.get('availableDocks', 0)
        station_dict['totalDocks']=station.get('totalDocks', 0)
        station_dict['availableBikes']=station.get('availableBikes',0)
        
        station_data.append(station_dict)
print(station_data) 


In [ ]:
import boto3

dynamodb = boto3.resource('dynamodb', region_name='us-east-2')



In [ ]:
table = dynamodb.create_table(
    TableName='Movies3',
    KeySchema=[
        {
            'AttributeName': 'year',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'title',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'year',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'title',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='Movies3')

# Print out some data about the table.
print(table.item_count)
print("Table status:", table.table_status)


In [ ]:
client = boto3.client('dynamodb', region_name='us-east-2')
response = client.list_tables()
print(response)

In [ ]:
data = {'trip_id': '12150150', 'starttime': '9/30/2016 23:57:50', 'stoptime': '10/1/2016 00:26:15', 'bikeid': '4481', 'tripduration': '1705', 'from_station_id': '302', 'from_station_name': 'Sheffield Ave & Wrightwood Ave', 'to_station_id': '492', 'to_station_name': 'Leavitt St & Addison St', 'usertype': 'Customer', 'gender': '', 'birthyear': ''}


In [ ]:
print(data)

In [ ]:
d = {dict_key:dict_value for dict_key, dict_value in data.items() if dict_value != ''}

In [ ]:
print(d)

In [ ]:
from collections import OrderedDict 

ordered = OrderedDict([('trip_id', '12150150'), ('starttime', '9/30/2016 23:57:50'), ('stoptime', '10/1/2016 00:26:15'), ('bikeid', '4481'), ('tripduration', '1705'), ('from_station_id', '302'), ('from_station_name', 'Sheffield Ave & Wrightwood Ave'), ('to_station_id', '492'), ('to_station_name', 'Leavitt St & Addison St'), ('usertype', 'Customer'), ('gender', ''), ('birthyear', '')])


In [ ]:
od = {dict_key:dict_value for dict_key, dict_value in ordered.items() if dict_value != ''}

In [ ]:
print(od)

In [ ]:
fuck = {dict_key:int(dict_value) for dict_key, dict_value in od.items()}

In [ ]:
class Trip:
    def __init__(
            self, trip_id, starttime, stoptime, bikeid,
            tripduration, from_station_id, from_station_name, to_station_id, to_station_name,
            usertype, gender, birthyear):
        self.trip_id = trip_id
        self.starttime = starttime
        self.stoptime = stoptime
        self.bikeid = bikeid
        self.tripduration = tripduration
        self.from_station_id = from_station_id
        self.from_station_name = from_station_name
        self.to_station_id = to_station_id
        self.to_station_name = to_station_name
        self.usertype = usertype
        self.gender = gender
        self.birthyear = birthyear

    @staticmethod
    def create_from_dict(lookup):
        return Trip(
            int(lookup['trip_id']),
            lookup['starttime'],
            lookup['stoptime'],
            int(lookup['bikeid']),
            int(lookup['tripduration']),
            int(lookup['from_station_id']),
            lookup['from_station_name'],
            int(lookup['to_station_id']),
            lookup['to_station_name'],
            lookup['usertype'],
            lookup['gender'],
            int(lookup['birthyear']) if lookup['birthyear'] else None
        )

In [ ]:
dict_row = Trip.create_from_dict(data)

In [ ]:
print(dict_row.__dict__)

In [ ]:
num = '12150150'

In [ ]:
print(int(num))

In [ ]:
import boto3
import json
import decimal
from boto3.dynamodb.conditions import Key, Attr

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

dynamodb = boto3.resource('dynamodb', region_name='us-east-2')

table = dynamodb.Table('StationData')

response = table.query(
    KeyConditionExpression=Key('station_id').eq(123)
)


count = response['Count']
print(response)
for i in response['Items']:
    print('staton_id:', i['station_id'], 'stationName:', i['stationName'], 'available_bike: ', i['availableBikes'], 'count:', count)


In [11]:
import boto3
import json
import decimal
from boto3.dynamodb.conditions import Key, Attr

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

dynamodb = boto3.resource('dynamodb', region_name='us-east-2')

table = dynamodb.Table('TripData')

response = table.query(
    KeyConditionExpression=Key('from_station_id').eq(2)
)


print(response)


ClientError: An error occurred (ValidationException) when calling the Query operation: Query condition missed key schema element: trip_id

In [5]:
from boto3 import resource
from boto3.dynamodb.conditions import Key

# The boto3 dynamoDB resource
dynamodb_resource = resource('dynamodb')

def scan_table(table_name, filter_key=None, filter_value=None):
    """
    Perform a scan operation on table.
    Can specify filter_key (col name) and its value to be filtered.
    """
    table = dynamodb_resource.Table(table_name)

    if filter_key and filter_value:
        filtering_exp = Key(filter_key).eq(filter_value)
        response = table.scan(FilterExpression=filtering_exp)
    else:
        response = table.scan()

    return response

In [12]:
scan_table('TripData', 'from_station_id', '2')

{'Items': [],
 'Count': 0,
 'ScannedCount': 3834,
 'LastEvaluatedKey': {'trip_id': Decimal('11944278')},
 'ResponseMetadata': {'RequestId': 'MDM3FR4QE98TS11SGVFG8C7D37VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 17 Jan 2019 03:28:25 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '90',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MDM3FR4QE98TS11SGVFG8C7D37VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1026058541'},
  'RetryAttempts': 0}}

In [ ]:
def scan_table_allpages(table_name, filter_key=None, filter_value=None):
    """
    Perform a scan operation on table. 
    Can specify filter_key (col name) and its value to be filtered. 
    This gets all pages of results. Returns list of items.
    """
    table = dynamodb_resource.Table(table_name)

    if filter_key and filter_value:
        filtering_exp = Key(filter_key).eq(filter_value)
        response = table.scan(FilterExpression=filtering_exp)
    else:
        response = table.scan()

    items = response['Items']
    while True:
        print(len(response['Items']))
        if response.get('LastEvaluatedKey'):
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
            items += response['Items']
        else:
            break

    return items


In [ ]:
scan_table_allpages('TripData', 'from_station_id', '487')


In [100]:
events = {'Records': [{'eventID': 'b307a282edf8b9b1df194583be22019e', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '2'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '30'}, 'latitude': {'N': '41.87647'}, 'station_id': {'N': '2'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Buckingham Fountain'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.62034'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.87647'}, 'station_id': {'N': '2'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Buckingham Fountain'}, 'availableBikes': {'N': '25'}, 'longitude': {'N': '-87.62034'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86249800000000005903908649', 'SizeBytes': 350, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'a7b8bc6ddfed703de41b06d31c4c9c6c', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '5'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.874053'}, 'station_id': {'N': '5'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'State St & Harrison St'}, 'availableBikes': {'N': '15'}, 'longitude': {'N': '-87.627716'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '8'}, 'latitude': {'N': '41.874053'}, 'station_id': {'N': '5'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'State St & Harrison St'}, 'availableBikes': {'N': '14'}, 'longitude': {'N': '-87.627716'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86249900000000005903908655', 'SizeBytes': 356, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '26238f9c3f247a85c93aba21b55dd0bd', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '4'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.856268'}, 'station_id': {'N': '4'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Burnham Harbor'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.613348'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '19'}, 'latitude': {'N': '41.856268'}, 'station_id': {'N': '4'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Burnham Harbor'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.613348'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250000000000005903908656', 'SizeBytes': 340, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'f819b40cec24edf561b88b811f25be22', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '3'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '33'}, 'latitude': {'N': '41.86722595682'}, 'station_id': {'N': '3'}, 'totalDocks': {'N': '55'}, 'stationName': {'S': 'Shedd Aquarium'}, 'availableBikes': {'N': '22'}, 'longitude': {'N': '-87.6153553902'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '31'}, 'latitude': {'N': '41.86722595682'}, 'station_id': {'N': '3'}, 'totalDocks': {'N': '55'}, 'stationName': {'S': 'Shedd Aquarium'}, 'availableBikes': {'N': '24'}, 'longitude': {'N': '-87.6153553902'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250100000000005903908657', 'SizeBytes': 350, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'a95681319fd71e0af1dfc827b3943731', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '6'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '24'}, 'latitude': {'N': '41.886976'}, 'station_id': {'N': '6'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Dusable Harbor'}, 'availableBikes': {'N': '15'}, 'longitude': {'N': '-87.612813'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '23'}, 'latitude': {'N': '41.886976'}, 'station_id': {'N': '6'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Dusable Harbor'}, 'availableBikes': {'N': '16'}, 'longitude': {'N': '-87.612813'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250200000000005903908658', 'SizeBytes': 340, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '16207dacc5df3dde6fc68aeac336fc6c', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '7'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '0'}, 'latitude': {'N': '41.88634906269'}, 'station_id': {'N': '7'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Field Blvd & South Water St'}, 'availableBikes': {'N': '19'}, 'longitude': {'N': '-87.6175165471'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.88634906269'}, 'station_id': {'N': '7'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Field Blvd & South Water St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.6175165471'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250300000000005903908659', 'SizeBytes': 375, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'eacbec4bf7c5c995bb6707dea8b59da1', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '15'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.858166'}, 'station_id': {'N': '15'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Racine Ave & 18th St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.656495'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '6'}, 'latitude': {'N': '41.858166'}, 'station_id': {'N': '15'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Racine Ave & 18th St'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.656495'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250400000000005903908660', 'SizeBytes': 352, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'e145b7e5eb33c6e0c9edc230cb05a9a2', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '18'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.883132'}, 'station_id': {'N': '18'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Wacker Dr & Washington St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.637321'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '16'}, 'latitude': {'N': '41.883132'}, 'station_id': {'N': '18'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Wacker Dr & Washington St'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.637321'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250500000000005903908661', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'b27c33b6ba8832ce2d2c8d536d28ce9d', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '16'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '8'}, 'latitude': {'N': '41.910337'}, 'station_id': {'N': '16'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Paulina Ave & North Ave'}, 'availableBikes': {'N': '7'}, 'longitude': {'N': '-87.670072'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.910337'}, 'station_id': {'N': '16'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Paulina Ave & North Ave'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.670072'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250600000000005903908662', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '23934559b1a8e59d4dcfad5150f31dd8', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '17'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.903119'}, 'station_id': {'N': '17'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Honore St & Division St'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.673935'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '15'}, 'latitude': {'N': '41.903119'}, 'station_id': {'N': '17'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Honore St & Division St'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.673935'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250700000000005903908663', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '6b59d5aef10be0a37fcbc5c1643bd557', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '11'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.76663823695'}, 'station_id': {'N': '11'}, 'totalDocks': {'N': '11'}, 'stationName': {'S': 'Jeffery Blvd & 71st St'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.5764501141'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.76663823695'}, 'station_id': {'N': '11'}, 'totalDocks': {'N': '11'}, 'stationName': {'S': 'Jeffery Blvd & 71st St'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.5764501141'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250800000000005903908664', 'SizeBytes': 366, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'ba6cb1eefe2ec0db8469b4fb800ce2a2', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '9'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.82879201994'}, 'station_id': {'N': '9'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Leavitt St & Archer Ave'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.6806044594'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.82879201994'}, 'station_id': {'N': '9'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Leavitt St & Archer Ave'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.6806044594'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86250900000000005903908665', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '36a3911115cbdcf2a8a62429a9b818c5', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '14'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.858086'}, 'station_id': {'N': '14'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Morgan St & 18th St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.651073'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.858086'}, 'station_id': {'N': '14'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Morgan St & 18th St'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.651073'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251000000000005903908666', 'SizeBytes': 350, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '69c842bf4f7b1b8d5edba417b87c8b1a', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '22'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.8694821'}, 'station_id': {'N': '22'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'May St & Taylor St'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.6554864'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '3'}, 'latitude': {'N': '41.8694821'}, 'station_id': {'N': '22'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'May St & Taylor St'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.6554864'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251100000000005903908667', 'SizeBytes': 352, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '782ffe9667abed5c00766f392acde4b3', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '28'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.91468'}, 'station_id': {'N': '28'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Larrabee St & Menomonee St'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.64332'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.91468'}, 'station_id': {'N': '28'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Larrabee St & Menomonee St'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.64332'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251200000000005903908668', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '41822b90afabf33537221b96f1aab646', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '27'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.91021'}, 'station_id': {'N': '27'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Larrabee St & North Ave'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.6435'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.91021'}, 'station_id': {'N': '27'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Larrabee St & North Ave'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.6435'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251300000000005903908669', 'SizeBytes': 356, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '207e334d115ed17c5aa2b899c0bd2cd9', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '24'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '0'}, 'latitude': {'N': '41.89186'}, 'station_id': {'N': '24'}, 'totalDocks': {'N': '13'}, 'stationName': {'S': 'Fairbanks Ct & Grand Ave'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.62062'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.89186'}, 'station_id': {'N': '24'}, 'totalDocks': {'N': '13'}, 'stationName': {'S': 'Fairbanks Ct & Grand Ave'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.62062'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251400000000005903908670', 'SizeBytes': 359, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '1b0e79a0337b7bbf1cf76aa628db3c85', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '12'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.7664094567'}, 'station_id': {'N': '12'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'South Shore Dr & 71st St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.5656875719'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.7664094567'}, 'station_id': {'N': '12'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'South Shore Dr & 71st St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.5656875719'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251500000000005903908671', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'a1bdc854941f9c81be341fc1bec1f495', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '19'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.868968'}, 'station_id': {'N': '19'}, 'totalDocks': {'N': '16'}, 'stationName': {'S': 'Loomis St & Taylor St (*)'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.659141'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.868968'}, 'station_id': {'N': '19'}, 'totalDocks': {'N': '16'}, 'stationName': {'S': 'Loomis St & Taylor St (*)'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.659141'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251600000000005903908672', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '5b2f99c14543e3ae24f63651fbd305d6', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '20'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.910522'}, 'station_id': {'N': '20'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Sheffield Ave & Kingsbury St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.653106'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.910522'}, 'station_id': {'N': '20'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Sheffield Ave & Kingsbury St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.653106'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251700000000005903908673', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'd842c54b9508d4ef7730b01bb00d4714', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '13'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '28'}, 'latitude': {'N': '41.932418'}, 'station_id': {'N': '13'}, 'totalDocks': {'N': '35'}, 'stationName': {'S': 'Wilton Ave & Diversey Pkwy'}, 'availableBikes': {'N': '7'}, 'longitude': {'N': '-87.652705'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '25'}, 'latitude': {'N': '41.932418'}, 'station_id': {'N': '13'}, 'totalDocks': {'N': '35'}, 'stationName': {'S': 'Wilton Ave & Diversey Pkwy'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.652705'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251800000000005903908674', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '2d71fd45a4a32457a75c29543ef46f23', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '23'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.902924'}, 'station_id': {'N': '23'}, 'totalDocks': {'N': '16'}, 'stationName': {'S': 'Orleans St & Elm St (*)'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.637715'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.902924'}, 'station_id': {'N': '23'}, 'totalDocks': {'N': '16'}, 'stationName': {'S': 'Orleans St & Elm St (*)'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.637715'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86251900000000005903908675', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'f7387ef3bee2292d687e97248607db01', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '25'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '5'}, 'latitude': {'N': '41.89766'}, 'station_id': {'N': '25'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Michigan Ave & Pearson St'}, 'availableBikes': {'N': '18'}, 'longitude': {'N': '-87.62351'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '8'}, 'latitude': {'N': '41.89766'}, 'station_id': {'N': '25'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Michigan Ave & Pearson St'}, 'availableBikes': {'N': '15'}, 'longitude': {'N': '-87.62351'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252000000000005903908676', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'b250ad50c9285bfcef1d766aea16b175', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '26'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '12'}, 'latitude': {'N': '41.890359'}, 'station_id': {'N': '26'}, 'totalDocks': {'N': '31'}, 'stationName': {'S': 'McClurg Ct & Illinois St'}, 'availableBikes': {'N': '19'}, 'longitude': {'N': '-87.617532'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '3'}, 'latitude': {'N': '41.890359'}, 'station_id': {'N': '26'}, 'totalDocks': {'N': '31'}, 'stationName': {'S': 'McClurg Ct & Illinois St'}, 'availableBikes': {'N': '28'}, 'longitude': {'N': '-87.617532'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252100000000005903908677', 'SizeBytes': 360, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '66f08bd896ed30d9c3230de40d3e5f61', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '21'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '12'}, 'latitude': {'N': '41.87772613'}, 'station_id': {'N': '21'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Aberdeen St & Jackson Blvd'}, 'availableBikes': {'N': '11'}, 'longitude': {'N': '-87.65478743'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.87772613'}, 'station_id': {'N': '21'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Aberdeen St & Jackson Blvd'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.65478743'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252200000000005903908678', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '3abbaff0459f62f65ee06f73e9bf42ca', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '29'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.90068'}, 'station_id': {'N': '29'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Noble St & Milwaukee Ave'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.6626'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.90068'}, 'station_id': {'N': '29'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Noble St & Milwaukee Ave'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.6626'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252300000000005903908679', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '65dad862b76fe6a3c7301ffc10d7f2a6', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '30'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.899643'}, 'station_id': {'N': '30'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Ashland Ave & Augusta Blvd'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.6677'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.899643'}, 'station_id': {'N': '30'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Ashland Ave & Augusta Blvd'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.6677'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252400000000005903908680', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '472ccf264a12b2e7fd624c72e71ff57a', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '31'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '19'}, 'latitude': {'N': '41.896776'}, 'station_id': {'N': '31'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Franklin St & Chicago Ave'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.635633'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.896776'}, 'station_id': {'N': '31'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Franklin St & Chicago Ave'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.635633'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252500000000005903908681', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '2060bb273f4f5bd30a9f87c55ff89f3f', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '35'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '32'}, 'latitude': {'N': '41.892278'}, 'station_id': {'N': '35'}, 'totalDocks': {'N': '47'}, 'stationName': {'S': 'Streeter Dr & Grand Ave'}, 'availableBikes': {'N': '15'}, 'longitude': {'N': '-87.612043'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.892278'}, 'station_id': {'N': '35'}, 'totalDocks': {'N': '47'}, 'stationName': {'S': 'Streeter Dr & Grand Ave'}, 'availableBikes': {'N': '29'}, 'longitude': {'N': '-87.612043'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252600000000005903908682', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '21d8cb9755d1b763f26d12bb0514308c', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '33'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '24'}, 'latitude': {'N': '41.877181'}, 'station_id': {'N': '33'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'State St & Van Buren St'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.627844'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '26'}, 'latitude': {'N': '41.877181'}, 'station_id': {'N': '33'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'State St & Van Buren St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.627844'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252700000000005903908683', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'd72940391f3e9df4db3bff979615f295', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '39'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '20'}, 'latitude': {'N': '41.87947235235'}, 'station_id': {'N': '39'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Wabash Ave & Adams St'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.6256886059'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '21'}, 'latitude': {'N': '41.87947235235'}, 'station_id': {'N': '39'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Wabash Ave & Adams St'}, 'availableBikes': {'N': '0'}, 'longitude': {'N': '-87.6256886059'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252800000000005903908684', 'SizeBytes': 363, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'c1e3286ba47b9cce8694f8fdf6c7571b', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '37'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '16'}, 'latitude': {'N': '41.8793563587'}, 'station_id': {'N': '37'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Dearborn St & Adams St'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.6297910363'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '17'}, 'latitude': {'N': '41.8793563587'}, 'station_id': {'N': '37'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Dearborn St & Adams St'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.6297910363'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86252900000000005903908685', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '84d63bf0b2630a881ccc837c62c88f0d', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '38'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '23'}, 'latitude': {'N': '41.88602082773'}, 'station_id': {'N': '38'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Clark St & Lake St'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.6308760584'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.88602082773'}, 'station_id': {'N': '38'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Clark St & Lake St'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.6308760584'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253000000000005903908686', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '4d8f108b40146b42c58ab29628c41ecc', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '41'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '1'}, 'latitude': {'N': '41.87207763285'}, 'station_id': {'N': '41'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Federal St & Polk St'}, 'availableBikes': {'N': '18'}, 'longitude': {'N': '-87.6295437729'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '5'}, 'latitude': {'N': '41.87207763285'}, 'station_id': {'N': '41'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Federal St & Polk St'}, 'availableBikes': {'N': '14'}, 'longitude': {'N': '-87.6295437729'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253100000000005903908687', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'fe8ee1a2cff94a17b6edb1d7a0a981d2', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '40'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.87934437346'}, 'station_id': {'N': '40'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'LaSalle St & Adams St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.6319852213'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '15'}, 'latitude': {'N': '41.87934437346'}, 'station_id': {'N': '40'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'LaSalle St & Adams St'}, 'availableBikes': {'N': '0'}, 'longitude': {'N': '-87.6319852213'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253200000000005903908688', 'SizeBytes': 363, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '72aa71c999455944c00c6cc1e138ccee', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '32'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.87464'}, 'station_id': {'N': '32'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Racine Ave & Congress Pkwy'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.65703'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.87464'}, 'station_id': {'N': '32'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Racine Ave & Congress Pkwy'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.65703'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253300000000005903908689', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '770f564747a8e8f19308df43b94b4015', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '36'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '36'}, 'latitude': {'N': '41.8777079559'}, 'station_id': {'N': '36'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Franklin St & Jackson Blvd'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.6353211408'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '37'}, 'latitude': {'N': '41.8777079559'}, 'station_id': {'N': '36'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Franklin St & Jackson Blvd'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.6353211408'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253400000000005903908690', 'SizeBytes': 372, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '36a52d000d791749cbaaf5649f8b9f32', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '45'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '6'}, 'latitude': {'N': '41.876243'}, 'station_id': {'N': '45'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Michigan Ave & Congress Pkwy'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.624426'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '8'}, 'latitude': {'N': '41.876243'}, 'station_id': {'N': '45'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Michigan Ave & Congress Pkwy'}, 'availableBikes': {'N': '7'}, 'longitude': {'N': '-87.624426'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253500000000005903908691', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '1a469d84e84a93064952fa224b2c4fcf', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '43'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '33'}, 'latitude': {'N': '41.8838927658'}, 'station_id': {'N': '43'}, 'totalDocks': {'N': '43'}, 'stationName': {'S': 'Michigan Ave & Washington St'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.6246491409'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '32'}, 'latitude': {'N': '41.8838927658'}, 'station_id': {'N': '43'}, 'totalDocks': {'N': '43'}, 'stationName': {'S': 'Michigan Ave & Washington St'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.6246491409'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253600000000005903908692', 'SizeBytes': 376, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'c6cb381f5bd4f3bb4205bfd00ec4a35d', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '34'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.9267559875'}, 'station_id': {'N': '34'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Cannon Dr & Fullerton Ave'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.6344287848'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.9267559875'}, 'station_id': {'N': '34'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Cannon Dr & Fullerton Ave'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.6344287848'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253700000000005903908693', 'SizeBytes': 370, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'b466ddffe507d5116fae52795312ac79', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '47'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '28'}, 'latitude': {'N': '41.889187'}, 'station_id': {'N': '47'}, 'totalDocks': {'N': '30'}, 'stationName': {'S': 'State St & Kinzie St'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.627754'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '26'}, 'latitude': {'N': '41.889187'}, 'station_id': {'N': '47'}, 'totalDocks': {'N': '30'}, 'stationName': {'S': 'State St & Kinzie St'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.627754'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253800000000005903908694', 'SizeBytes': 352, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'cfc8535e41da63b6427966ed5f42e15d', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '46'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '16'}, 'latitude': {'N': '41.89993001'}, 'station_id': {'N': '46'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Wells St & Walton St'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.63443007'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.89993001'}, 'station_id': {'N': '46'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Wells St & Walton St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.63443007'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86253900000000005903908695', 'SizeBytes': 356, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '9a6503262a24c887469fef64ac878b27', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '44'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '25'}, 'latitude': {'N': '41.8847302006'}, 'station_id': {'N': '44'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'State St & Randolph St'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.6277335692'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '12'}, 'latitude': {'N': '41.8847302006'}, 'station_id': {'N': '44'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'State St & Randolph St'}, 'availableBikes': {'N': '15'}, 'longitude': {'N': '-87.6277335692'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254000000000005903908696', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '3809bc30ea13de6252444872b89a3a94', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '50'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '4'}, 'latitude': {'N': '41.8759326655'}, 'station_id': {'N': '50'}, 'totalDocks': {'N': '35'}, 'stationName': {'S': 'Clark St & Congress Pkwy'}, 'availableBikes': {'N': '31'}, 'longitude': {'N': '-87.6305845355'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.8759326655'}, 'station_id': {'N': '50'}, 'totalDocks': {'N': '35'}, 'stationName': {'S': 'Clark St & Congress Pkwy'}, 'availableBikes': {'N': '25'}, 'longitude': {'N': '-87.6305845355'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254100000000005903908697', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '423fa2c8ceca875077f3e7c126c05093', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '48'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '40'}, 'latitude': {'N': '41.897764'}, 'station_id': {'N': '48'}, 'totalDocks': {'N': '43'}, 'stationName': {'S': 'Larrabee St & Kingsbury St'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.642884'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '42'}, 'latitude': {'N': '41.897764'}, 'station_id': {'N': '48'}, 'totalDocks': {'N': '43'}, 'stationName': {'S': 'Larrabee St & Kingsbury St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.642884'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254200000000005903908698', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'ae185b5cdd8ef4410bca9f2882d7ff82', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '49'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '39'}, 'latitude': {'N': '41.881319815'}, 'station_id': {'N': '49'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Dearborn St & Monroe St'}, 'availableBikes': {'N': '0'}, 'longitude': {'N': '-87.6295209193'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '33'}, 'latitude': {'N': '41.881319815'}, 'station_id': {'N': '49'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Dearborn St & Monroe St'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.6295209193'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254300000000005903908699', 'SizeBytes': 365, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'e17c7307de7af98ee233dd11d8fdcc2d', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '51'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.884576228'}, 'station_id': {'N': '51'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Clark St & Randolph St'}, 'availableBikes': {'N': '20'}, 'longitude': {'N': '-87.63188991'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '24'}, 'latitude': {'N': '41.884576228'}, 'station_id': {'N': '51'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Clark St & Randolph St'}, 'availableBikes': {'N': '15'}, 'longitude': {'N': '-87.63188991'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254400000000005903908700', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'ccbd9f4810225c6b87d72d616c4415fb', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '42'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '6'}, 'latitude': {'N': '41.852619'}, 'station_id': {'N': '42'}, 'totalDocks': {'N': '11'}, 'stationName': {'S': 'Wabash Ave & Cermak Rd'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.626488'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.852619'}, 'station_id': {'N': '42'}, 'totalDocks': {'N': '11'}, 'stationName': {'S': 'Wabash Ave & Cermak Rd'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.626488'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254500000000005903908701', 'SizeBytes': 356, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'e9f2bfcab7c0f3a3bf89c6db112a9d0b', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '52'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '33'}, 'latitude': {'N': '41.886024'}, 'station_id': {'N': '52'}, 'totalDocks': {'N': '35'}, 'stationName': {'S': 'Michigan Ave & Lake St'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.624117'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '32'}, 'latitude': {'N': '41.886024'}, 'station_id': {'N': '52'}, 'totalDocks': {'N': '35'}, 'stationName': {'S': 'Michigan Ave & Lake St'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.624117'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254600000000005903908702', 'SizeBytes': 356, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '046b7bb94d7ce3546113fba71e45fea4', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '53'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.894722'}, 'station_id': {'N': '53'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Wells St & Huron St'}, 'availableBikes': {'N': '14'}, 'longitude': {'N': '-87.634362'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.894722'}, 'station_id': {'N': '53'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Wells St & Huron St'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.634362'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254700000000005903908703', 'SizeBytes': 350, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'fd933b04beac838cf998e283de1ae93f', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '54'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '5'}, 'latitude': {'N': '41.896362458'}, 'station_id': {'N': '54'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Ogden Ave & Chicago Ave'}, 'availableBikes': {'N': '14'}, 'longitude': {'N': '-87.6540612729'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.896362458'}, 'station_id': {'N': '54'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Ogden Ave & Chicago Ave'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.6540612729'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254800000000005903908720', 'SizeBytes': 366, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '7356fb36e120f336a1398c1c364416b8', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '55'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '16'}, 'latitude': {'N': '41.867324'}, 'station_id': {'N': '55'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Halsted St & Roosevelt Rd'}, 'availableBikes': {'N': '7'}, 'longitude': {'N': '-87.648625'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '23'}, 'latitude': {'N': '41.867324'}, 'station_id': {'N': '55'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Halsted St & Roosevelt Rd'}, 'availableBikes': {'N': '0'}, 'longitude': {'N': '-87.648625'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86254900000000005903908721', 'SizeBytes': 361, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '30d0c3f6897742915eeac7392255a72c', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '57'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '3'}, 'latitude': {'N': '41.8671177825'}, 'station_id': {'N': '57'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Clinton St & Roosevelt Rd'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.6410879593'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '3'}, 'latitude': {'N': '41.8671177825'}, 'station_id': {'N': '57'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Clinton St & Roosevelt Rd'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.6410879593'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255000000000005903908722', 'SizeBytes': 370, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '32c1ca8d7a5facbb637a09a423420cbb', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '58'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '20'}, 'latitude': {'N': '41.916017'}, 'station_id': {'N': '58'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Marshfield Ave & Cortland St'}, 'availableBikes': {'N': '3'}, 'longitude': {'N': '-87.668879'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '21'}, 'latitude': {'N': '41.916017'}, 'station_id': {'N': '58'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Marshfield Ave & Cortland St'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.668879'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255100000000005903908723', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '110d63a23872a8af0f84751fb3c039c5', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '59'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '3'}, 'latitude': {'N': '41.867227'}, 'station_id': {'N': '59'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Wabash Ave & Roosevelt Rd'}, 'availableBikes': {'N': '20'}, 'longitude': {'N': '-87.625961'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.867227'}, 'station_id': {'N': '59'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Wabash Ave & Roosevelt Rd'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.625961'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255200000000005903908724', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'a860b58be939867ccc61762d9efb4624', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '56'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '8'}, 'latitude': {'N': '41.888716036'}, 'station_id': {'N': '56'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Desplaines St & Kinzie St'}, 'availableBikes': {'N': '18'}, 'longitude': {'N': '-87.6444478533'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.888716036'}, 'station_id': {'N': '56'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Desplaines St & Kinzie St'}, 'availableBikes': {'N': '13'}, 'longitude': {'N': '-87.6444478533'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255300000000005903908725', 'SizeBytes': 370, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '3fbc5da050a8df755cb4375b4e11fd1e', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '60'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '17'}, 'latitude': {'N': '41.9105780349'}, 'station_id': {'N': '60'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Dayton St & North Ave'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.6494219288'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.9105780349'}, 'station_id': {'N': '60'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Dayton St & North Ave'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.6494219288'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255400000000005903908726', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '828b3d5e1b72c8d22842526613280a5f', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '61'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.907655'}, 'station_id': {'N': '61'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Wood St & Milwaukee Ave'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.672552'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '5'}, 'latitude': {'N': '41.907655'}, 'station_id': {'N': '61'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Wood St & Milwaukee Ave'}, 'availableBikes': {'N': '14'}, 'longitude': {'N': '-87.672552'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255500000000005903908727', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '0de9125b1291ab05827256e29aa5fd42', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '68'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '5'}, 'latitude': {'N': '41.875885'}, 'station_id': {'N': '68'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Clinton St & Tilden St'}, 'availableBikes': {'N': '18'}, 'longitude': {'N': '-87.640795'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.875885'}, 'station_id': {'N': '68'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Clinton St & Tilden St'}, 'availableBikes': {'N': '13'}, 'longitude': {'N': '-87.640795'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255600000000005903908728', 'SizeBytes': 356, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'b309189bedc9cb3a1a0187bf4cc5a0e1', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '66'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.885637'}, 'station_id': {'N': '66'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Clinton St & Lake St'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.641823'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '0'}, 'latitude': {'N': '41.885637'}, 'station_id': {'N': '66'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Clinton St & Lake St'}, 'availableBikes': {'N': '22'}, 'longitude': {'N': '-87.641823'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255700000000005903908729', 'SizeBytes': 351, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'ab0be52e69413164a2cdf1cab4e96753', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '72'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '6'}, 'latitude': {'N': '41.860384'}, 'station_id': {'N': '72'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Wabash Ave & 16th St'}, 'availableBikes': {'N': '32'}, 'longitude': {'N': '-87.625813'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.860384'}, 'station_id': {'N': '72'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Wabash Ave & 16th St'}, 'availableBikes': {'N': '29'}, 'longitude': {'N': '-87.625813'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255800000000005903908730', 'SizeBytes': 352, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '21384902883b2a0ecbf27c19c69d0607', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '62'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '16'}, 'latitude': {'N': '41.8513751729'}, 'station_id': {'N': '62'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'McCormick Place'}, 'availableBikes': {'N': '11'}, 'longitude': {'N': '-87.6188346489'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.8513751729'}, 'station_id': {'N': '62'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'McCormick Place'}, 'availableBikes': {'N': '14'}, 'longitude': {'N': '-87.6188346489'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86255900000000005903908731', 'SizeBytes': 350, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '3106b9ef784c0722f0b9422d4676a943', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '67'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '22'}, 'latitude': {'N': '41.9256018819'}, 'station_id': {'N': '67'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Sheffield Ave & Fullerton Ave'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.6537080423'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '23'}, 'latitude': {'N': '41.9256018819'}, 'station_id': {'N': '67'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Sheffield Ave & Fullerton Ave'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.6537080423'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256000000000005903908732', 'SizeBytes': 378, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'a51bf2808cbf222d27e29ee579a1f55a', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '69'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.9093960065'}, 'station_id': {'N': '69'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Damen Ave & Pierce Ave'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.6776919292'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.9093960065'}, 'station_id': {'N': '69'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Damen Ave & Pierce Ave'}, 'availableBikes': {'N': '13'}, 'longitude': {'N': '-87.6776919292'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256100000000005903908733', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'ca58c1e37a4218cbd4a05f72b9516876', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '71'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.8854833079'}, 'station_id': {'N': '71'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Morgan St & Lake St'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.6523048564'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '17'}, 'latitude': {'N': '41.8854833079'}, 'station_id': {'N': '71'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Morgan St & Lake St'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.6523048564'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256200000000005903908734', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'e74a42a047a2de2a76f746f1c3ac9a41', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '80'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '12'}, 'latitude': {'N': '41.880419'}, 'station_id': {'N': '80'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Aberdeen St & Monroe St'}, 'availableBikes': {'N': '7'}, 'longitude': {'N': '-87.655519'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.880419'}, 'station_id': {'N': '80'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Aberdeen St & Monroe St'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.655519'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256300000000005903908735', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'c1e8db13824b06b7f04429ed4c37922c', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '73'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.880422'}, 'station_id': {'N': '73'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Jefferson St & Monroe St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.642746'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '15'}, 'latitude': {'N': '41.880422'}, 'station_id': {'N': '73'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Jefferson St & Monroe St'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.642746'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256400000000005903908736', 'SizeBytes': 360, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'dac06e4ac90eea6a3fe187c60f31d75e', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '74'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.893626'}, 'station_id': {'N': '74'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Kingsbury St & Erie St'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.641527'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '5'}, 'latitude': {'N': '41.893626'}, 'station_id': {'N': '74'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Kingsbury St & Erie St'}, 'availableBikes': {'N': '18'}, 'longitude': {'N': '-87.641527'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256500000000005903908737', 'SizeBytes': 356, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '7e43e1c03614ab300874bafe97218c3a', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '76'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '31'}, 'latitude': {'N': '41.880958'}, 'station_id': {'N': '76'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Lake Shore Dr & Monroe St'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.616743'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '29'}, 'latitude': {'N': '41.880958'}, 'station_id': {'N': '76'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Lake Shore Dr & Monroe St'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.616743'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256600000000005903908738', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'b1219f5c9552b3b2be14044dd6eaf818', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '86'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.89637337'}, 'station_id': {'N': '86'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Eckhart Park'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.66098386'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.89637337'}, 'station_id': {'N': '86'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Eckhart Park'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.66098386'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256700000000005903908739', 'SizeBytes': 340, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '60ba6b56968267a1cf1eb61cce42af29', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '84'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '6'}, 'latitude': {'N': '41.891578'}, 'station_id': {'N': '84'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Milwaukee Ave & Grand Ave'}, 'availableBikes': {'N': '13'}, 'longitude': {'N': '-87.648384'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '1'}, 'latitude': {'N': '41.891578'}, 'station_id': {'N': '84'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Milwaukee Ave & Grand Ave'}, 'availableBikes': {'N': '17'}, 'longitude': {'N': '-87.648384'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256800000000005903908740', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'e9a89379081f591ce26ec244ebc48383', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '77'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '17'}, 'latitude': {'N': '41.882242'}, 'station_id': {'N': '77'}, 'totalDocks': {'N': '31'}, 'stationName': {'S': 'Clinton St & Madison St'}, 'availableBikes': {'N': '14'}, 'longitude': {'N': '-87.641066'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '2'}, 'latitude': {'N': '41.882242'}, 'station_id': {'N': '77'}, 'totalDocks': {'N': '31'}, 'stationName': {'S': 'Clinton St & Madison St'}, 'availableBikes': {'N': '29'}, 'longitude': {'N': '-87.641066'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86256900000000005903908741', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'c5f6547de468b268dc129b66b4a2f75e', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '81'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '32'}, 'latitude': {'N': '41.884241'}, 'station_id': {'N': '81'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Daley Center Plaza'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.629634'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '19'}, 'latitude': {'N': '41.884241'}, 'station_id': {'N': '81'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Daley Center Plaza'}, 'availableBikes': {'N': '19'}, 'longitude': {'N': '-87.629634'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257000000000005903908742', 'SizeBytes': 348, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'c9460e65c76694357222cc0573e96c5b', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '85'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '15'}, 'latitude': {'N': '41.90096039'}, 'station_id': {'N': '85'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Michigan Ave & Oak St'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.62377664'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.90096039'}, 'station_id': {'N': '85'}, 'totalDocks': {'N': '23'}, 'stationName': {'S': 'Michigan Ave & Oak St'}, 'availableBikes': {'N': '14'}, 'longitude': {'N': '-87.62377664'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257100000000005903908743', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'e92bd6fa55b32b7a90634985ddbebfdf', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '75'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '3'}, 'latitude': {'N': '41.877245'}, 'station_id': {'N': '75'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Canal St & Jackson Blvd'}, 'availableBikes': {'N': '36'}, 'longitude': {'N': '-87.639366'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '19'}, 'latitude': {'N': '41.877245'}, 'station_id': {'N': '75'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Canal St & Jackson Blvd'}, 'availableBikes': {'N': '19'}, 'longitude': {'N': '-87.639366'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257200000000005903908744', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '23f298818bb3f0111aa7c5167e1c6eac', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '87'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '2'}, 'latitude': {'N': '41.92556258'}, 'station_id': {'N': '87'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Racine Ave & Fullerton Ave'}, 'availableBikes': {'N': '17'}, 'longitude': {'N': '-87.65840426'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.92556258'}, 'station_id': {'N': '87'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Racine Ave & Fullerton Ave'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.65840426'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257300000000005903908793', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '476f9134ec4d1a03eb945ea4a271c923', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '88'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.884069'}, 'station_id': {'N': '88'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Racine Ave & Randolph St'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.656853'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.884069'}, 'station_id': {'N': '88'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Racine Ave & Randolph St'}, 'availableBikes': {'N': '10'}, 'longitude': {'N': '-87.656853'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257400000000005903908794', 'SizeBytes': 360, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '61174bd9062869a2d768727c43ee8274', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '92'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.894556'}, 'station_id': {'N': '92'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Carpenter St & Huron St'}, 'availableBikes': {'N': '11'}, 'longitude': {'N': '-87.653449'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '8'}, 'latitude': {'N': '41.894556'}, 'station_id': {'N': '92'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Carpenter St & Huron St'}, 'availableBikes': {'N': '11'}, 'longitude': {'N': '-87.653449'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257500000000005903908795', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '3653f01c8095f63cfa2a8f262e2a6af9', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '91'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '2'}, 'latitude': {'N': '41.88338'}, 'station_id': {'N': '91'}, 'totalDocks': {'N': '31'}, 'stationName': {'S': 'Clinton St & Washington Blvd'}, 'availableBikes': {'N': '28'}, 'longitude': {'N': '-87.64117'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '2'}, 'latitude': {'N': '41.88338'}, 'station_id': {'N': '91'}, 'totalDocks': {'N': '31'}, 'stationName': {'S': 'Clinton St & Washington Blvd'}, 'availableBikes': {'N': '27'}, 'longitude': {'N': '-87.64117'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257600000000005903908796', 'SizeBytes': 368, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'c762b605d57e70804bba841bcd04ea5b', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '95'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.778881'}, 'station_id': {'N': '95'}, 'totalDocks': {'N': '11'}, 'stationName': {'S': 'Stony Island Ave & 64th St'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.586647'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.778881'}, 'station_id': {'N': '95'}, 'totalDocks': {'N': '11'}, 'stationName': {'S': 'Stony Island Ave & 64th St'}, 'availableBikes': {'N': '2'}, 'longitude': {'N': '-87.586647'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257700000000005903908797', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'c705b2a398c88f02866ff851171e29c1', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '96'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.88461618962'}, 'station_id': {'N': '96'}, 'totalDocks': {'N': '22'}, 'stationName': {'S': 'Desplaines St & Randolph St'}, 'availableBikes': {'N': '7'}, 'longitude': {'N': '-87.6445705849'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '17'}, 'latitude': {'N': '41.88461618962'}, 'station_id': {'N': '96'}, 'totalDocks': {'N': '22'}, 'stationName': {'S': 'Desplaines St & Randolph St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.6445705849'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257800000000005903908798', 'SizeBytes': 376, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '8575b18ef8d7bc4b2337a99e9f801cf1', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '93'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.913688'}, 'station_id': {'N': '93'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Sheffield Ave & Willow St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.652855'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '3'}, 'latitude': {'N': '41.913688'}, 'station_id': {'N': '93'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Sheffield Ave & Willow St'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.652855'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86257900000000005903908799', 'SizeBytes': 362, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'dcc1f520b6963fdb24c583e4cc65aa19', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '94'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.918306'}, 'station_id': {'N': '94'}, 'totalDocks': {'N': '31'}, 'stationName': {'S': 'Clark St & Armitage Ave'}, 'availableBikes': {'N': '13'}, 'longitude': {'N': '-87.636282'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.918306'}, 'station_id': {'N': '94'}, 'totalDocks': {'N': '31'}, 'stationName': {'S': 'Clark St & Armitage Ave'}, 'availableBikes': {'N': '18'}, 'longitude': {'N': '-87.636282'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258000000000005903908800', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'b4de2d09f2b767ec038380c3bb897fb5', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '98'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.882664'}, 'station_id': {'N': '98'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'LaSalle St & Washington St'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.63253'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.882664'}, 'station_id': {'N': '98'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'LaSalle St & Washington St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.63253'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258100000000005903908801', 'SizeBytes': 364, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '58a0e7605e34a4e56cb366c890b4dd33', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '100'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '31'}, 'latitude': {'N': '41.888243'}, 'station_id': {'N': '100'}, 'totalDocks': {'N': '35'}, 'stationName': {'S': 'Orleans St & Merchandise Mart Plaza'}, 'availableBikes': {'N': '0'}, 'longitude': {'N': '-87.63639'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '26'}, 'latitude': {'N': '41.888243'}, 'station_id': {'N': '100'}, 'totalDocks': {'N': '35'}, 'stationName': {'S': 'Orleans St & Merchandise Mart Plaza'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.63639'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258200000000005903908802', 'SizeBytes': 381, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '4619f92640a5847f2048d19b0977067d', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '97'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '42'}, 'latitude': {'N': '41.865312'}, 'station_id': {'N': '97'}, 'totalDocks': {'N': '55'}, 'stationName': {'S': 'Field Museum'}, 'availableBikes': {'N': '13'}, 'longitude': {'N': '-87.617867'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '37'}, 'latitude': {'N': '41.865312'}, 'station_id': {'N': '97'}, 'totalDocks': {'N': '55'}, 'stationName': {'S': 'Field Museum'}, 'availableBikes': {'N': '18'}, 'longitude': {'N': '-87.617867'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258300000000005903908803', 'SizeBytes': 336, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '6b09fbaae8d37a3492ab68ddc21e39de', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '90'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '31'}, 'latitude': {'N': '41.8810317'}, 'station_id': {'N': '90'}, 'totalDocks': {'N': '47'}, 'stationName': {'S': 'Millennium Park'}, 'availableBikes': {'N': '16'}, 'longitude': {'N': '-87.62408432'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '38'}, 'latitude': {'N': '41.8810317'}, 'station_id': {'N': '90'}, 'totalDocks': {'N': '47'}, 'stationName': {'S': 'Millennium Park'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.62408432'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258400000000005903908804', 'SizeBytes': 346, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'e2da760094e721b4cb3402bd9c087b0e', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '99'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.89257'}, 'station_id': {'N': '99'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Lake Shore Dr & Ohio St'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.614492'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '13'}, 'latitude': {'N': '41.89257'}, 'station_id': {'N': '99'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Lake Shore Dr & Ohio St'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.614492'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258500000000005903908805', 'SizeBytes': 358, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'd2af0b26ad6b5dd5c80c706dbd66b4fc', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '89'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '0'}, 'latitude': {'N': '41.874618'}, 'station_id': {'N': '89'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Financial Pl & Congress Pkwy (Temp)'}, 'availableBikes': {'N': '15'}, 'longitude': {'N': '-87.632882'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '6'}, 'latitude': {'N': '41.874618'}, 'station_id': {'N': '89'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Financial Pl & Congress Pkwy (Temp)'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.632882'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258600000000005903908806', 'SizeBytes': 381, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '9cdb0a4a932cbc9f190a6a7967e9b40f', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '102'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.77345849948'}, 'station_id': {'N': '102'}, 'totalDocks': {'N': '11'}, 'stationName': {'S': 'Stony Island Ave & 67th St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.5853397391'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.77345849948'}, 'station_id': {'N': '102'}, 'totalDocks': {'N': '11'}, 'stationName': {'S': 'Stony Island Ave & 67th St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.5853397391'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258700000000005903908807', 'SizeBytes': 377, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'eccb5a406841c158c1df33aba09c0269', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '101'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.78101637196'}, 'station_id': {'N': '101'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': '63rd St Beach'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.5761197602'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.78101637196'}, 'station_id': {'N': '101'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': '63rd St Beach'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.5761197602'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258800000000005903908808', 'SizeBytes': 351, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '71fd7e4ace9ba53cbd3591f967b1f006', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '106'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '19'}, 'latitude': {'N': '41.897448'}, 'station_id': {'N': '106'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'State St & Pearson St'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.628722'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.897448'}, 'station_id': {'N': '106'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'State St & Pearson St'}, 'availableBikes': {'N': '9'}, 'longitude': {'N': '-87.628722'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86258900000000005903908809', 'SizeBytes': 357, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '2e708ab55ffbc5adac08c98f444a1e1c', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '103'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '10'}, 'latitude': {'N': '41.87146651779'}, 'station_id': {'N': '103'}, 'totalDocks': {'N': '16'}, 'stationName': {'S': 'Clinton St & Polk St (*)'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.6409491327'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.87146651779'}, 'station_id': {'N': '103'}, 'totalDocks': {'N': '16'}, 'stationName': {'S': 'Clinton St & Polk St (*)'}, 'availableBikes': {'N': '5'}, 'longitude': {'N': '-87.6409491327'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86259000000000005903908810', 'SizeBytes': 373, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '90011d08cb1e8f80ecaf92974d5a700d', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '110'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '16'}, 'latitude': {'N': '41.893992'}, 'station_id': {'N': '110'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Dearborn St & Erie St'}, 'availableBikes': {'N': '11'}, 'longitude': {'N': '-87.629318'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.893992'}, 'station_id': {'N': '110'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Dearborn St & Erie St'}, 'availableBikes': {'N': '18'}, 'longitude': {'N': '-87.629318'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86259100000000005903908811', 'SizeBytes': 357, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'd620ff0f59356c1c47cef8f199e7d6d0', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '109'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '18'}, 'latitude': {'N': '41.874754'}, 'station_id': {'N': '109'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': '900 W Harrison St'}, 'availableBikes': {'N': '1'}, 'longitude': {'N': '-87.649807'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '19'}, 'latitude': {'N': '41.874754'}, 'station_id': {'N': '109'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': '900 W Harrison St'}, 'availableBikes': {'N': '0'}, 'longitude': {'N': '-87.649807'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86259200000000005903908812', 'SizeBytes': 348, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'f7f6e9ed9a4d0e2efa350e18707932bc', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '108'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '12'}, 'latitude': {'N': '41.87184'}, 'station_id': {'N': '108'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Halsted St & Polk St'}, 'availableBikes': {'N': '7'}, 'longitude': {'N': '-87.64664'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '15'}, 'latitude': {'N': '41.87184'}, 'station_id': {'N': '108'}, 'totalDocks': {'N': '19'}, 'stationName': {'S': 'Halsted St & Polk St'}, 'availableBikes': {'N': '4'}, 'longitude': {'N': '-87.64664'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86259300000000005903908813', 'SizeBytes': 355, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '6f5a310b265ab9b95bc46a3de14fe89a', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '107'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '8'}, 'latitude': {'N': '41.878287'}, 'station_id': {'N': '107'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Desplaines St & Jackson Blvd'}, 'availableBikes': {'N': '19'}, 'longitude': {'N': '-87.643909'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '11'}, 'latitude': {'N': '41.878287'}, 'station_id': {'N': '107'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Desplaines St & Jackson Blvd'}, 'availableBikes': {'N': '16'}, 'longitude': {'N': '-87.643909'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86259400000000005903908814', 'SizeBytes': 371, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '0e2ccd87a0b765c248958c646edb1640', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '111'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '20'}, 'latitude': {'N': '41.894666'}, 'station_id': {'N': '111'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Sedgwick St & Huron St'}, 'availableBikes': {'N': '7'}, 'longitude': {'N': '-87.638437'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '21'}, 'latitude': {'N': '41.894666'}, 'station_id': {'N': '111'}, 'totalDocks': {'N': '27'}, 'stationName': {'S': 'Sedgwick St & Huron St'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.638437'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86259500000000005903908815', 'SizeBytes': 359, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': 'c1de14ae34ed3977f3fbc5f3b536688c', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '112'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '7'}, 'latitude': {'N': '41.883668'}, 'station_id': {'N': '112'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Green St & Randolph St'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.64867'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.883668'}, 'station_id': {'N': '112'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Green St & Randolph St'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.64867'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86259600000000005903908816', 'SizeBytes': 359, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}, {'eventID': '4251a67653dcbbd676aa6c5bdc4ac986', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '113'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '3'}, 'latitude': {'N': '41.91844'}, 'station_id': {'N': '113'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Bissell St & Armitage Ave'}, 'availableBikes': {'N': '12'}, 'longitude': {'N': '-87.65222'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '9'}, 'latitude': {'N': '41.91844'}, 'station_id': {'N': '113'}, 'totalDocks': {'N': '15'}, 'stationName': {'S': 'Bissell St & Armitage Ave'}, 'availableBikes': {'N': '6'}, 'longitude': {'N': '-87.65222'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86259700000000005903908817', 'SizeBytes': 365, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}, 'eventSourceARN': 'arn:aws:dynamodb:us-east-2:378083460762:table/StationData/stream/2019-01-14T03:54:43.186'}]}

In [103]:
for record in events['Records']:
        #print(type(record),record)
        print(record['dynamodb']['Keys']['station_id'].get('N'))
        #print(record['eventName'])
        #print(record['Keys'])
        #print("DynamoDB Record: " + json.dumps(record['dynamodb'], indent=2))
    #return 'Successfully processed {} records.'.format(len(event['Records']))
print(event['Records'][0]['dynamodb']['Keys']['station_id'].get('N'))

2
5
4
3
6
7
15
18
16
17
11
9
14
22
28
27
24
12
19
20
13
23
25
26
21
29
30
31
35
33
39
37
38
41
40
32
36
45
43
34
47
46
44
50
48
49
51
42
52
53
54
55
57
58
59
56
60
61
68
66
72
62
67
69
71
80
73
74
76
86
84
77
81
85
75
87
88
92
91
95
96
93
94
98
100
97
90
99
89
102
101
106
103
110
109
108
107
111
112
113
2


In [83]:
print(type(response['Records']),response['Records'])

KeyError: 'Records'

In [ ]:
import boto3
import json
from boto3.dynamodb.conditions import Key

from_station_list = []
for record in events['Records']:
        from_station_list.append(int(record['dynamodb']['Keys']['station_id'].get('N')))

dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
table = dynamodb.Table('TripDataSample')

for i in from_station_list:
    fe = Key('from_station_id').eq(i)
    #pe = "from_station_id, from_station_name"


    response = table.query(
    KeyConditionExpression=fe,
    IndexName='from_station_id-index'
    )


    items = response['Items']

    while 'LastEvaluatedKey' in response:
        response = table.query(
            #ProjectionExpression=pe,
            KeyConditionExpression=fe,
            IndexName='from_station_id-index',
            ExclusiveStartKey=response['LastEvaluatedKey']
            )
        items += response['Items']

    from_count_dict = {}
    from_count_dict['from_station_id']=i
    from_count_dict['from_count']=len(items)
    print(from_count_dict)

    #dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
    #table = dynamodb.Table('from_station_count')
    #response = table.put_item(Item=from_count_dict)


In [120]:
import boto3
import json
from boto3.dynamodb.conditions import Key

def lambda_handler(event, context):
    print('event:', event)
    from_station_list = []
    for record in event['Records']:
            from_station_list.append(int(record['dynamodb']['Keys']['station_id'].get('N')))    

    for i in from_station_list:
        fe = Key('from_station_id').eq(i)
        #pe = "from_station_id, from_station_name"   
    
        dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
        tbl_trip_data = dynamodb.Table('TripDataSample')    
        
        response = tbl_trip_data.query(
            KeyConditionExpression=fe,
            IndexName='from_station_id-index'
            )   

        items = response['Items']   

        while 'LastEvaluatedKey' in response:
            response = tbl_trip_data.query(
                KeyConditionExpression=fe,
                IndexName='from_station_id-index',
                ExclusiveStartKey=response['LastEvaluatedKey']
                )
            items += response['Items']  

        from_count_dict = {}
        from_count_dict['from_station_id']=i
        from_count_dict['from_count']=len(items)
        print(from_count_dict)  

        tbl_from_count = dynamodb.Table('from_station_count')
        response = tbl_from_count.put_item(Item=from_count_dict)

In [122]:
lambda_handler(events, None)

event: {'Records': [{'eventID': 'b307a282edf8b9b1df194583be22019e', 'eventName': 'MODIFY', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'us-east-2', 'dynamodb': {'ApproximateCreationDateTime': 1547784281.0, 'Keys': {'station_id': {'N': '2'}}, 'NewImage': {'load_datetime': {'S': '2019-01-17T21:04:40.434471'}, 'availableDocks': {'N': '30'}, 'latitude': {'N': '41.87647'}, 'station_id': {'N': '2'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Buckingham Fountain'}, 'availableBikes': {'N': '8'}, 'longitude': {'N': '-87.62034'}, 'status': {'S': 'IN_SERVICE'}}, 'OldImage': {'load_datetime': {'S': '2019-01-14T21:43:19.708498'}, 'availableDocks': {'N': '14'}, 'latitude': {'N': '41.87647'}, 'station_id': {'N': '2'}, 'totalDocks': {'N': '39'}, 'stationName': {'S': 'Buckingham Fountain'}, 'availableBikes': {'N': '25'}, 'longitude': {'N': '-87.62034'}, 'status': {'S': 'IN_SERVICE'}}, 'SequenceNumber': '86249800000000005903908649', 'SizeBytes': 350, 'StreamViewType': 'NEW_A

{'from_station_id': 2, 'from_count': 40}
{'from_station_id': 5, 'from_count': 39}
{'from_station_id': 4, 'from_count': 25}
{'from_station_id': 3, 'from_count': 66}
{'from_station_id': 6, 'from_count': 25}
{'from_station_id': 7, 'from_count': 31}
{'from_station_id': 15, 'from_count': 29}
{'from_station_id': 18, 'from_count': 123}
{'from_station_id': 16, 'from_count': 35}
{'from_station_id': 17, 'from_count': 36}
{'from_station_id': 11, 'from_count': 0}
{'from_station_id': 9, 'from_count': 3}
{'from_station_id': 14, 'from_count': 25}
{'from_station_id': 22, 'from_count': 31}
{'from_station_id': 28, 'from_count': 37}
{'from_station_id': 27, 'from_count': 30}
{'from_station_id': 24, 'from_count': 110}
{'from_station_id': 12, 'from_count': 0}
{'from_station_id': 19, 'from_count': 51}
{'from_station_id': 20, 'from_count': 50}
{'from_station_id': 13, 'from_count': 89}
{'from_station_id': 23, 'from_count': 21}
{'from_station_id': 25, 'from_count': 47}
{'from_station_id': 26, 'from_count': 89}


In [75]:
import boto3
import json
from boto3.dynamodb.conditions import Key

def lambda_handler(event, context):
    print('event:', event)
    from_station_list = []
    for record in event['Records']:
            from_station_list.append(int(record['dynamodb']['Keys']['station_id'].get('N')))    

    
    for i in from_station_list:
        fe = Key('from_station_id').eq(i)
        pe = "from_station_id, from_station_name"   
    
        dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
        tbl_trip_data = dynamodb.Table('TripDataSample')    
        
        response = tbl_trip_data.scan(
            FilterExpression=fe,
            ProjectionExpression=pe,
            )   

        items = response['Items']   

        while 'LastEvaluatedKey' in response:
            response = tbl_trip_data.scan(
                ProjectionExpression=pe,
                FilterExpression=fe,
                ExclusiveStartKey=response['LastEvaluatedKey']
                )
            items += response['Items']  

        from_count_dict = {}
        from_count_dict['from_station_id']=i
        from_count_dict['from_count']=len(items)
        print(from_count_dict)  

        tbl_from_count = dynamodb.Table('from_station_count')
        response = tbl_from_count.put_item(Item=from_count_dict)

In [41]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
from boto3.dynamodb.conditions import Key, Attr

station_id = 2

dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
table = dynamodb.Table('TripDataSample')

fe = Key('gender').eq('Female')
#pe = "from_station_id, from_station_name"


response = table.scan(
    FilterExpression=fe,
    #ProjectionExpression=pe,
    )

items = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(
        #ProjectionExpression=pe,
        FilterExpression=fe,
        ExclusiveStartKey=response['LastEvaluatedKey']
        )
    items += response['Items']
print(len(items))
#print('station_id:', station_id, 'count:', len(items))

1891


In [63]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
from boto3.dynamodb.conditions import Key, Attr
import statistics

station_id = 2

dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
table = dynamodb.Table('TripDataSample')

fe = Key('trip_id').gt(0)
pe = "tripduration"

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)


response = table.scan(
    FilterExpression=fe,
    ProjectionExpression=pe,
    )

items = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(
        ProjectionExpression=pe,
        FilterExpression=fe,
        ExclusiveStartKey=response['LastEvaluatedKey']
        )
    items += response['Items']
#data = json.dumps(items[:5], cls=DecimalEncoder) 
#ave_dur = statistics.mean(lambda x : x[data]

x = [i['tripduration'] for i in items]
#print(x)
#print(items)   
ave_dur = statistics.mean(x)
print(ave_dur)

762.9306389121021043834992023


In [52]:
query_table('TripData','from_station_id', 2)

ClientError: An error occurred (ValidationException) when calling the Query operation: Query condition missed key schema element: trip_id